# Itália #

In [1]:
import os
import sys
from osgeo import ogr
os.environ['QT_QPA_PLATFORM'] = 'offscreen'
sys.path.insert(0,'/usr/local/share/qgis/python')
from qgis.core import *
QgsApplication.setPrefixPath("/usr/local", True) 
from qgis.gui import *
from qgis import processing
from IPython.display import Image
from qgis.PyQt.QtGui import QColor, QImage
from qgis.PyQt.QtCore import QSize, QBuffer, QIODevice

qgs = QgsApplication([], True)
qgs.initQgis()
print(QgsApplication.showSettings())
project = QgsProject.instance()

Application state:
QGIS_PREFIX_PATH env var:		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS
Prefix:		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS
Plugin Path:		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../PlugIns/qgis
Package Data Path:	/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources
Active Theme Name:	
Active Theme Path:	/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources/resources/themes//icons/
Default Theme Path:	:/images/themes/default/
SVG Search Paths:	/usr/local/../Resources/svg/
		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources/svg/
		/Users/dianamartins/Library/Application Support/profiles/default/svg/
User DB Path:	/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources/resources/qgis.db
Auth DB Path:	/Users/dianamartins/Library/Application Support/profiles/default/qgis-auth.db



In [2]:
italy_map = "/Users/dianamartins/Documents/GitHub/Epidemiologia/Trabalho_final/Dados/ITA_adm/ITA_adm1.shp"
vlayer = QgsVectorLayer(italy_map, "italy_map", "ogr")
if not vlayer.isValid():
    print("Layer failed to load!")
else:
    QgsProject.instance().addMapLayer(vlayer)
    print("Layer loaded!")

Layer loaded!


In [3]:
csv_path = "file:///Users/dianamartins/Documents/GitHub/Epidemiologia/Trabalho_final/Dados/casos_italia_atual.csv?delimiter=,"
csv_layer = QgsVectorLayer(csv_path, 'Data', 'delimitedtext')

if not csv_layer.isValid():
    print('Layer failed to load!')
else:
    QgsProject.instance().addMapLayer(csv_layer)
    print("Layer loaded !")

Layer loaded !


#### Juntar as camadas ####

In [4]:
joinName='codice_regione'
targetName='ID_1'

joinObject=QgsVectorLayerJoinInfo()
joinObject.setJoinFieldName(joinName)
joinObject.setTargetFieldName(targetName)
joinObject.setJoinLayerId(csv_layer.id())
joinObject.setUsingMemoryCache(True)
joinObject.setJoinLayer(csv_layer)
flag = vlayer.addJoin(joinObject)



In [11]:
novospositivos_layer = vlayer.clone()
novospositivos_layer.setName("Novos positivos")

for field in deaths_layer.fields():
    print("Column "+field.name()+" Load")

Column ID_0 Load
Column ISO Load
Column NAME_0 Load
Column ID_1 Load
Column NAME_1 Load
Column TYPE_1 Load
Column ENGTYPE_1 Load
Column NL_NAME_1 Load
Column VARNAME_1 Load
Column Data_data Load
Column Data_stato Load
Column Data_denominazione_regione Load
Column Data_lat Load
Column Data_long Load
Column Data_ricoverati_con_sintomi Load
Column Data_terapia_intensiva Load
Column Data_totale_ospedalizzati Load
Column Data_isolamento_domiciliare Load
Column Data_totale_positivi Load
Column Data_variazione_totale_positivi Load
Column Data_nuovi_positivi Load
Column Data_dimessi_guariti Load
Column Data_deceduti Load
Column Data_totale_casi Load
Column Data_tamponi Load
Column Data_casi_testati Load
Column Data_note_it Load
Column Data_note_en Load


In [16]:
from qgis.PyQt import QtGui


myOpacity = 1
myTargetField = 'Data_nuovi_positivi'
myRangeList = []
# Make our first symbol and range...
myMin = 0
myMax = 10
myLabel = '0-10'
myColour = QtGui.QColor('#ffffff')
mySymbol = QgsSymbol.defaultSymbol(novospositivos_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)
#now make another symbol and range...
myMin = 10
myMax = 30
myLabel = '10 - 30'
myColour = QtGui.QColor('#ffbbbb')
mySymbol = QgsSymbol.defaultSymbol(novospositivos_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)
#now make another symbol and range...
myMin = 30
myMax = 50
myLabel = '30 - 50'
myColour = QtGui.QColor('#ff8888')
mySymbol = QgsSymbol.defaultSymbol(novospositivos_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)
#now make another symbol and range...
myMin = 50
myMax = 100
myLabel = '50 - 100'
myColour = QtGui.QColor('#ff4444')
mySymbol = QgsSymbol.defaultSymbol(novospositivos_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)
#now make another symbol and range...
myMin = 100
myMax = 400
myLabel = '100 - 400'
myColour = QtGui.QColor('#ff0000')
mySymbol = QgsSymbol.defaultSymbol(novospositivos_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)


novospositivos_layer.setRenderer(myRenderer)



In [19]:
xt = novospositivos_layer.extent()
width  = 1024
height = int(width*xt.height()/xt.width())

project = QgsProject().instance()
project.addMapLayers([cases_layer])
manager = project.layoutManager()
layoutName = 'novos_casos'
layouts_list = manager.printLayouts()
for layout in layouts_list:
    if layout.name() == layoutName:
        manager.removeLayout(layout)
        
layout = QgsPrintLayout(project)
layout.initializeDefaults()
layout.setName(layoutName)
manager.addLayout(layout)

pc = layout.pageCollection()
pc.page(0).attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))


map = QgsLayoutItemMap(layout)
map.setRect(20,20,20,20)
map.setLayers([novospositivos_layer])



ms = QgsMapSettings()
ms.setLayers([novospositivos_layer])
ms.setExtent(xt)
map.setExtent(xt)


layout.addLayoutItem(map)



map.attemptMove(QgsLayoutPoint(0,0,QgsUnitTypes.LayoutMillimeters))
map.attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))


map.setBackgroundColor(QColor(250,250,250))




legend = QgsLayoutItemLegend(layout)

legend.setReferencePoint(6)
legend.setScale(3)
legend.attemptMove(QgsLayoutPoint(0,height,QgsUnitTypes.LayoutMillimeters))

layout.addLayoutItem(legend)
legend.setLegendFilterByMapEnabled(True)

legend.setLinkedMap(map)

exporter = QgsLayoutExporter(manager.layoutByName("novos_casos"))

image = exporter.renderPageToImage(0)
image = exporter.renderPageToImage(0)

from IPython.display import Image
imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image.save( imgbuf,"PNG" )
image.save('/Users/dianamartins/Documents/GitHub/Epidemiologia/Trabalho_final/Imagens/novos_casosatual',"PNG")
Image( imgbuf.data() )

False